In [ ]:
import time, os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import layers, metrics, losses, callbacks, regularizers
from tensorflow.python.client import device_lib
from sklearn.model_selection import train_test_split
from kaggle_datasets import KaggleDatasets
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',titleweight='bold', titlesize=18, titlepad=10)
plt.rc('image', cmap='magma')

In [ ]:
# Baixar conjunto de dados
!unzip -q -n ../input/galaxy-zoo-the-galaxy-challenge/images_training_rev1.zip -d ../temp/
!unzip -q -n ../input/galaxy-zoo-the-galaxy-challenge/images_test_rev1.zip -d ../temp/     
labels_pd = pd.read_csv('../input/galaxy-zoo-the-galaxy-challenge/training_solutions_rev1.zip',compression='zip')

for dirname, _, filenames in os.walk('../input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
print("Dataset loading done!")

In [ ]:
# Hardware de inicialização (CPU / GPU / TPU)
strategy = tf.distribute.get_strategy()
#estratégia de distribuição padrão no Tensorflow. Funciona com CPU e GPU única.
print('Running on CPU/GPU')
    

In [ ]:
# Pré-processamento de conjunto de dados
BATCH_SIZE = 64
AUTOTUNE = tf.data.AUTOTUNE
def get_image_and_features(image_path, training=True):
    image = tf.io.decode_image(tf.io.read_file(image_path), dtype=tf.dtypes.float32)
    image = tf.image.resize_with_crop_or_pad(image, 100, 100) 
    if training:
        label = tf.strings.split(image_path, os.path.sep)[3] # take the galaxy number from image path
        galaxyID = int(tf.strings.substr(label, 0, tf.strings.length(label)-4)) # remove .jpg extension
        galaxy_row = labels_pd.loc[labels_pd['GalaxyID'] == galaxyID]
        features = tf.cast(galaxy_row.iloc[0,1:].values, tf.float32)
        return image, features
    else:
        return image
    
def dataset_preprocessing(images_path, training=True, tpu=False):
      
    images_path_ds = tf.data.Dataset.from_tensor_slices(images_path)
    if training:
        dataset = images_path_ds.map(lambda x: tf.py_function(func=get_image_and_features, inp=[x], Tout=(tf.float32,tf.float32)),num_parallel_calls=AUTOTUNE)
    else:
        dataset = images_path_ds.map(lambda x: tf.py_function(func=get_image_and_features, inp=[x,False], Tout=tf.float32),num_parallel_calls=AUTOTUNE)
        
    #dataset = images_path_ds.map(get_image_and_features, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(2048) if training else dataset
    dataset = dataset.batch(BATCH_SIZE) # Set batch size
    dataset = dataset.prefetch(AUTOTUNE) # Add dataset prefetch() operations to reduce read latency while training the model
    return dataset

images_path = tf.io.gfile.glob('../temp/images_training_rev1/*')
seed = 54
tf.random.set_seed(seed)
images_path = tf.random.shuffle(images_path)
samples_size = len(images_path)
print("Number of total samples: {}".format(samples_size))

# Dividir conjuntos de validação de trem
splitIndex = int(np.floor(samples_size*0.8))
images_path_train = images_path[:splitIndex]
images_path_val = images_path[splitIndex:]
     
train_dataset = dataset_preprocessing(images_path_train)
print('Number of training batches: %d' % tf.data.experimental.cardinality(train_dataset))

val_dataset = dataset_preprocessing(images_path_val)
print('Number of validation batches: %d' % tf.data.experimental.cardinality(val_dataset))

print("Dataset preprocessing done!")

In [ ]:
# Plot example
def plot_example(dataset, rows=2, cols=4):
    fig, axes = plt.subplots(rows, cols, figsize=(12, 6))
    images = list(dataset.take(1).as_numpy_iterator())[0][0]
    labels = list(dataset.take(1).as_numpy_iterator())[0][1]
    for i in range(rows):
        for j in range(cols):
            axes[i,j].grid(False)
            axes[i,j].axis('off')
            axes[i,j].imshow(images[cols*i+j, :])
    plt.show()
    return images[0].shape, labels[0].shape
image_shape, features_num = plot_example(train_dataset)
print("Images shape is: {}".format(image_shape))
print("Features length is: {}".format(features_num))

In [ ]:
# Plot example 2
def plot_example(dataset, rows=2, cols=2):
    fig, axes = plt.subplots(rows, cols, figsize=(24, 12))
    images = list(dataset.take(1).as_numpy_iterator())[0][0]
    labels = list(dataset.take(1).as_numpy_iterator())[0][1]
    for i in range(rows):
        for j in range(cols):
            axes[i,j].grid(False)
            axes[i,j].axis('off')
            axes[i,j].imshow(images[cols*i+j, :])
    plt.show()
    return images[0].shape, labels[0].shape
image_shape, features_num = plot_example(train_dataset)
print("Images shape is: {}".format(image_shape))
print("Features length is: {}".format(features_num))

In [ ]:
# Modelo
def create_model(input_shape, use_augmentation=False):
    model = tf.keras.models.Sequential(name="galaxyClassifier", layers=[
        layers.Conv2D(filters=16,kernel_size=(7, 7),activation='relu',input_shape=input_shape),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.1),
        layers.Conv2D(filters=32,kernel_size=(6, 6),activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.1),
        layers.Conv2D(filters=64,kernel_size=(5, 5),activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.1),
        layers.Conv2D(filters=128,kernel_size=(3, 3),activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.1),
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(37, activation='sigmoid')
    ])

    if use_augmentation:
        data_augmentation = tf.keras.models.Sequential(name='augmentation', layers=[
            layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical", input_shape=input_shape),
            layers.experimental.preprocessing.RandomRotation(0.3),
            layers.experimental.preprocessing.RandomZoom(height_factor=0.2, width_factor=0.2),
            layers.experimental.preprocessing.RandomContrast(0.05)])
        model = tf.keras.models.Sequential(name="galaxyClassifier", layers=[
            data_augmentation,
            model])
        
    return model

with strategy.scope(): 
    image_shape = (100,100,3)
    model = create_model(image_shape, True)
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3) # Optimizer
    loss_func = losses.MeanSquaredError() # Loss function
    model.compile(loss=loss_func, optimizer=optimizer, metrics=[tf.keras.metrics.RootMeanSquaredError()])
    
model.summary()

In [ ]:
# Treinamento
num_epochs = 10
verbose = True
# Chamadas de retorno
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=7, min_lr=1e-5)
def decay_schedule(epoch, lr):
    return lr * 0.8 if (epoch % 10 == 0) and (epoch != 0) else lr
lr_scheduler = callbacks.LearningRateScheduler(decay_schedule)
early_stop = callbacks.EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=7)
checkpoint = callbacks.ModelCheckpoint('best_model', save_best_only=True, monitor='val_accuracy', mode='max')
callbacksInUse = []

print('------- Training -------')
start = time.time()
history = model.fit(train_dataset, validation_data=val_dataset, epochs=num_epochs, callbacks=callbacksInUse, use_multiprocessing=True, verbose=verbose)
end = time.time()
print("Total training took {:.2f} hours.".format((end - start)/3600))

# Traçar curvas de aprendizagem
metrics = history.history
fig, axes = plt.subplots(1, 2, figsize=(12,6))
axes[0].plot(metrics['root_mean_squared_error'], label='train_accuracy')
axes[0].plot(metrics['val_root_mean_squared_error'], label='val_accuracy')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('RMSE')
axes[0].legend(loc='upper right')
axes[1].plot(metrics['loss'], label='train_loss')
axes[1].plot(metrics['val_loss'], label='val_loss')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend(loc='upper right')
plt.show()

In [ ]:
# Avaliação
results = model.evaluate(val_dataset, verbose=1)
print("Test RMSE: {:.3f}".format(results[1]))

In [ ]:
# Savar modelo
outPath = model.name+".h5"
model.save(outPath)
print("Model is saved: {}".format(outPath))

#model.load_weights('../input/models/galaxyClassifier1.h5')
print("Model is loaded succesfully!")

In [ ]:
# Predição
test_images_path = tf.io.gfile.glob('../temp/images_test_rev1/*')
test_dataset = dataset_preprocessing(test_images_path,False)
print('Number of test batches: %d' % tf.data.experimental.cardinality(test_dataset))
print('------- Prediction -------')
result = model.predict(test_dataset, verbose=1)
print("result shape: {}".format(result.shape))


## VGG19

In [ ]:
import numpy as np #álgebra Linear
import pandas as pd #processamento de dados, E / S de arquivo CSV

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

### Descompactando os Dados

In [ ]:
import os
import zipfile

def unzip(file, destination):
    print('Unzipping to', destination)
    with zipfile.ZipFile(file, 'r') as zip_ref:
        zip_ref.extractall(destination)

base_dir = "/kaggle/tmp/"
train_images_path = os.path.join(base_dir, "images_training_rev1")
test_images_path = os.path.join(base_dir, "images_test_rev1")

if not os.path.exists(base_dir):
    unzip('/kaggle/input/galaxy-zoo-the-galaxy-challenge/images_training_rev1.zip', base_dir)
    unzip('/kaggle/input/galaxy-zoo-the-galaxy-challenge/images_test_rev1.zip', base_dir)
    unzip('/kaggle/input/galaxy-zoo-the-galaxy-challenge/training_solutions_rev1.zip', base_dir)

for dirname, _, filenames in os.walk(base_dir):
    for filename in filenames[:5]:
        print(os.path.join(dirname, filename))


In [ ]:
import pandas as pd

def append_ext(filename):
    """ Appends `.jpg` file extension to a filename """
    return f"{filename}.jpg"

train_sol = pd.read_csv("/kaggle/tmp/training_solutions_rev1.csv")
train_sol["GalaxyID"] = train_sol["GalaxyID"].apply(append_ext)
train_sol.head()

### Preparação dos Dados

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_dataframe(
    dataframe=train_sol,
    directory=train_images_path,
    x_col="GalaxyID",
    y_col=["Class1.1", "Class1.2", "Class1.3"],
    clases=['Early type', 'Spiral', 'Artifact'],
    subset="training",
    batch_size=32,
    shuffle=False,
    class_mode="raw",
    target_size=(224,224)
)

valid_generator = datagen.flow_from_dataframe(
    dataframe=train_sol,
    directory=train_images_path,
    x_col="GalaxyID",
    y_col=["Class1.1", "Class1.2", "Class1.3"],
    subset="validation",
    batch_size=32,
    shuffle=False,
    class_mode="raw",
    target_size=(224,224)
)

train_steps = np.ceil(train_generator.samples / train_generator.batch_size)
val_steps = np.ceil(valid_generator.samples / valid_generator.batch_size)

### Modelo

In [ ]:
from tensorflow.keras import layers, Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import ResNet50

def build_model(num_classes):

    pre_trained_model = VGG19(input_shape=(224,224,3),
                            weights='imagenet',
                            include_top=False) 

    # Achatar a camada de saída para 1 dimensão
    x = layers.Flatten()(pre_trained_model.output)


    # Adicionar uma camada totalmente conectada com 1024 unidades ocultas e ativação ReLU (x2)
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dense(1024, activation='relu')(x)


    # Adicionar uma taxa de abandono de 0,2
    x = layers.Dropout(0.2)(x)


    # Adicionar uma camada softmax final para classificação
    output = layers.Dense(num_classes, activation='softmax')(x)

    # Definir o modelo
    model = Model( pre_trained_model.input, output )

    return model, pre_trained_model

model, pre_trained_model = build_model(3)
print(model.summary())

for layer in pre_trained_model.layers:
    layer.trainable = False

print(model.summary())

### Compilando o modelo

In [ ]:
from tensorflow.keras.optimizers import Adam
#from keras import backend as K

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=1e-3),
              metrics=['accuracy'])

In [ ]:
from matplotlib import pyplot as plt

def plot_history(history):
    # Recupera uma lista de resultados de precisão em conjuntos de dados de treinamento e 
    #...teste para cada período de treinamento

    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    # Recupera uma lista de resultados da lista de dados de treinamento e teste
    # conjuntos para cada época de treinamento
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    # Obter o número de épocas
    epochs = range(len(acc))

    # Traçar treino e precisão de validação por época
    plt.figure(dpi=150)
    plt.plot(epochs, acc)
    plt.plot(epochs, val_acc)
    plt.ylabel('Accuracy')
    plt.ylim([0,1])
    plt.legend( ('training', 'validation') )
    plt.figure()

    # Traçar treinamento e perda de validação por época
    plt.figure(dpi=150)
    plt.plot(epochs, loss)
    plt.plot(epochs, val_loss)
    plt.ylabel('Loss')
    plt.title('Training and validation loss')

### Treinamento de modelo

In [ ]:
tf_history = model.fit_generator(train_generator,
#                                  steps_per_epoch=256,
                                 steps_per_epoch=train_steps,
                                 epochs=5,
                                 validation_data=valid_generator,
#                                  validation_steps=256,
                                 validation_steps=val_steps,
                                 verbose=2)

plot_history( tf_history )

In [ ]:
for layer in pre_trained_model.layers[-5:]:
    layer.trainable = True
    
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=1e-4),
              metrics=['accuracy'])
    
tf_history = model.fit_generator(train_generator,
#                                  steps_per_epoch=256,
                                 steps_per_epoch=train_steps,
                                 epochs=3,
                                 validation_data=valid_generator,
#                                  validation_steps=256,
                                 validation_steps=val_steps,
                                 verbose=2)

plot_history( tf_history )

In [ ]:
for layer_idx, layer in enumerate(model.layers):
  # verifica a camada convolucional
  if not 'convolutional' in str(layer.__class__):
    continue
  print(layer_idx, layer.name, layer.output.shape)

visualization_model = Model(model.input, model.layers[1].output)

In [ ]:
visualization_model = Model(model.input, model.layers[1].output)

In [ ]:
def show(img):
  '''display image'''
  plt.figure(figsize=(8,8))
  plt.grid(False)
  plt.axis('Off')
  plt.imshow(img)
  plt.show()

next_data = valid_generator.next()
img = next_data[0][0]
show(img)

# expand dimensions so that it fakes a batch containing a single sample
img = np.expand_dims(img, axis=0)

print(f"Ground truth:\t   {next_data[1][0]}")
print(f"Model prediction: {model.predict(img)}")

In [ ]:
feature_maps = visualization_model.predict(img)

In [ ]:
square = 8
fig = plt.gcf()
fig.set_size_inches(square*2,square*2)
idx = 1
for _ in range(square):
  for _ in range(square):
    sp = plt.subplot(square, square, idx)
    sp.axis('Off')
    sp.title.set_text(str(idx-1))
    plt.imshow(feature_maps[0, :, :, idx-1])
    idx += 1

# plt.show()

In [ ]:
layer_indices = [1, 2, 4, 5, 7, 8, 9, 10, 12, 13, 14, 15, 17, 18, 19, 20]
visualization_model = Model( model.input, [model.layers[idx].output for idx in layer_indices] )

In [ ]:
#@title Plot feature maps
plt.imshow(img[0])
plt.axis('Off')
plt.title('Input Image')
plt.show()

square = 8
feature_maps = visualization_model.predict(img)
for layer_idx, fmap in enumerate(feature_maps):
    fig = plt.gcf()
    fig.set_size_inches(square*2, square*2)
    fig.suptitle(model.layers[layer_indices[layer_idx]].name)
    idx = 0
    
    for _ in range(square):
        for _ in range(2):
            fm = fmap[0, :, :, idx]
            sp = plt.subplot(square, square, idx+1)
            sp.axis('Off')
            sp.title.set_text(str(idx))
            plt.imshow(fm)
            idx += 1

    plt.show()

In [ ]:
#@title Plot average feature maps
plt.imshow(img[0])
plt.axis('Off')
plt.title('Input Image')
plt.show()

fig=plt.figure(figsize=(150, 150))
for layer_idx, fmap in enumerate(feature_maps[:5]):
    sp = fig.add_subplot(1, len(feature_maps), layer_idx+1)
    sp.axis('Off')
    sp.title.set_text(model.layers[ layer_indices[layer_idx] ].name)
    plt.imshow(np.squeeze(fmap.mean(axis=-1)))
    
fig=plt.figure(figsize=(150, 150))
for layer_idx, fmap in enumerate(feature_maps[5:10]):
    sp = fig.add_subplot(1, len(feature_maps), layer_idx+1)
    sp.axis('Off')
    sp.title.set_text(model.layers[ layer_indices[layer_idx] ].name)
    plt.imshow(np.squeeze(fmap.mean(axis=-1)))
    
fig=plt.figure(figsize=(150, 150))
for layer_idx, fmap in enumerate(feature_maps[10:15]):
    sp = fig.add_subplot(1, len(feature_maps), layer_idx+1)
    sp.axis('Off')
    sp.title.set_text(model.layers[ layer_indices[layer_idx] ].name)
    plt.imshow(np.squeeze(fmap.mean(axis=-1)))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

Y_pred = model.predict_generator(valid_generator, val_steps)
y_pred = np.argmax(Y_pred, axis=1)
y_true = np.argmax(valid_generator.labels, axis=1)

print('Confusion Matrix')
print(confusion_matrix(y_true, y_pred))

print('Classification Report')
target_names = ['Early type', 'Spiral', 'Artifact']
print(classification_report(y_true, y_pred, target_names=target_names))